<a href="https://colab.research.google.com/github/supplient/bachelor_design/blob/no_equal/Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment Setup

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [3]:
!git clone https://github.com/supplient/bachelor_design.git
import os
os.chdir("bachelor_design")
!pwd

fatal: destination path 'bachelor_design' already exists and is not an empty directory.
/content/bachelor_design


In [4]:
!git checkout no_equal

Branch 'no_equal' set up to track remote branch 'no_equal' from 'origin'.
Switched to a new branch 'no_equal'


In [5]:
!git pull

Already up to date.


# Mount GDriver
Perpare Colab environment.

In [6]:
from driver_amount import addh

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
[Colab] Using address head: /gdrive/My Drive


In [7]:
!pip install keras_bert

  Created wheel for keras-bert: filename=keras_bert-0.84.0-cp36-none-any.whl size=36139 sha256=6b3a119e3883ff58dc0e284aa892f4b1d635706f63896c7d64b287fc0742ca12
  Stored in directory: /root/.cache/pip/wheels/1f/59/04/12e95257aebfd27f7edaaf65ab7dd57b5f6cadfb183f1a4ccd
  Created wheel for keras-transformer: filename=keras_transformer-0.37.0-cp36-none-any.whl size=12942 sha256=627343abf88d3981e490af85e24a21763c15629427f3ce706e30abcdf834ecc9
  Stored in directory: /root/.cache/pip/wheels/2a/f9/31/2a3289e948852ce0dd3fcd94c34bbc7eb9628842cb7110a87b
  Created wheel for keras-pos-embd: filename=keras_pos_embd-0.11.0-cp36-none-any.whl size=7554 sha256=a907ce183132898141b60ce823e200dafe64f8f37ed96a1062088108edf62ae8
  Stored in directory: /root/.cache/pip/wheels/5b/a1/a0/ce6b1d49ba1a9a76f592e70cf297b05c96bc9f418146761032
  Created wheel for keras-multi-head: filename=keras_multi_head-0.27.0-cp36-none-any.whl size=15611 sha256=53466a715d13f2b4d0897d055cf52bf1188e27afe03d3f5f7926667037cf53b9
  Stor

In [8]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-pkupvjqr
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-pkupvjqr
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101064 sha256=80d531decf883cb8f66e48fa8aecda850d9628a6713935d9de6b34a1a8f294de
  Stored in directory: /tmp/pip-ephem-wheel-cache-6mciy38v/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [0]:
import os
os.chdir("/usr/local/lib/python3.6/dist-packages/keras_contrib/layers/")
lines = []
with open("crf.py", "r") as fd:
  lines = fd.readlines()
lines[515] = "            mask2 = K.cast(K.concatenate([mask, K.cast(K.zeros_like(mask[:, :1]), mask.dtype)], axis=1),\n"
with open("crf.py", "w") as fd:
  fd.writelines(lines)

os.chdir("/content")

In [0]:
os.chdir("bachelor_design")

# Prepare Input

In [0]:
from cut_and_tag import cut_and_tag
import preprocess
import config

In [12]:
cut_seqs, char_seqs, tag_seqs = cut_and_tag(
    addh + config.DATA_PATH,
    addh + config.STOPWORDS_PATH
)
char_seqs, tag_seqs = preprocess.shuffle_twin(
    char_seqs,
    tag_seqs
)
token_id_seqs, segment_seqs, tag_id_seqs, tag_vocab = preprocess.preprocess(
    char_seqs, 
    tag_seqs,
    addh + config.BERT_VOCAB_PATH,
    SEQ_LEN=config.SEQ_LEN
    )

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.942 seconds.
Prefix dict has been built successfully.
Using TensorFlow backend.


In [0]:
# Split train and text
train_num = int(len(token_id_seqs) * 0.9)

test_token_id_seqs = token_id_seqs[train_num:]
test_segment_seqs = segment_seqs[train_num:]
test_tag_id_seqs = tag_id_seqs[train_num:]

train_token_id_seqs = token_id_seqs[0:train_num]
train_segment_seqs = segment_seqs[0:train_num]
train_tag_id_seqs = tag_id_seqs[0:train_num]

In [0]:
import json
with open(addh + config.TAG_VOCAB_PATH, "w") as fd:
    json.dump(tag_vocab, fd)

In [15]:
tag_vocab

{'': 0,
 'B-EI': 2,
 'B-EIF': 8,
 'B-EO': 4,
 'B-EQ': 10,
 'B-ILF': 6,
 'I-EI': 3,
 'I-EIF': 9,
 'I-EO': 5,
 'I-EQ': 11,
 'I-ILF': 7,
 'O': 1}

# Build Model
using BERT

In [0]:
import keras
import keras_bert 

In [17]:
bert_model = keras_bert.load_trained_model_from_checkpoint(
    # config_file
    addh + config.BERT_CONFIG_PATH, 
    # checkpoint file
    addh + config.BERT_CHECKPOINT_PATH, 
    # settings
    training=False, # Not train the whole model. Ignore NSP and MLM
    trainable=True

)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
from keras_contrib.layers import CRF

In [0]:
input_token = keras.layers.Input(shape=(config.SEQ_LEN,))
input_segment = keras.layers.Input(shape=(config.SEQ_LEN,))

bert_output = bert_model([input_token, input_segment])

crf_model = CRF(len(tag_vocab), sparse_target=True)

output = crf_model(bert_output)

model = keras.models.Model([input_token, input_segment], output)

In [20]:
model.compute_output_shape([(1, config.SEQ_LEN), (1, config.SEQ_LEN)])

(1, 512, 12)

In [21]:
bert_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, 512)          0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, 512)          0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 512, 768), ( 91812096    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 512, 768)     1536        Input-Segment[0][0]              
____________________________________________________________________________________________

In [22]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512)          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 512)          0                                            
__________________________________________________________________________________________________
model_2 (Model)                 (None, 512, 768)     177262848   input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
crf_1 (CRF)                     (None, 512, 12)      9396        model_2[1][0]              

# Fine-tuning

In [23]:
model.compile(
    optimizer=keras.optimizers.Adam(lr=0.00004),
    loss=crf_model.loss_function,
    metrics=[crf_model.accuracy]
)

/usr/local/lib/python3.6/dist-packages/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/usr/local/lib/python3.6/dist-packages/keras_contrib/layers/crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy
  warnings.warn('CRF.accuracy is deprecated and it '


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
import epoch_checkpoint
epoch_callback = epoch_checkpoint.EpochCheckpoint(
    addh + config.MODEL_PATH,
    addh + config.TRAIN_REC_PATH,
    1,
    [test_token_id_seqs, test_segment_seqs],
    test_tag_id_seqs,
    tag_vocab
    )

In [0]:
batch_size = 16
completed = False
import tensorflow as tf

In [27]:
while not completed:
  try:
    model.fit(
      x=[train_token_id_seqs, train_segment_seqs],
      y=[train_tag_id_seqs], 
      batch_size=batch_size, 
      epochs=30,
      verbose=1,
      validation_data=[[test_token_id_seqs, test_segment_seqs], [test_tag_id_seqs]],
      callbacks=[epoch_callback]
    )
    completed = True
  except tf.errors.ResourceExhaustedError as e:
    batch_size = int(batch_size/2)
    print("Batch Size too large, turn to " + str(batch_size))
    if batch_size < 1:
      raise e


Train on 7005 samples, validate on 779 samples
Epoch 1/30
Batch Size too large, turn to 8
Train on 7005 samples, validate on 779 samples
Epoch 1/30
Batch Size too large, turn to 4
Train on 7005 samples, validate on 779 samples
Epoch 1/30
  84/7005 [..............................] - ETA: 1:13:22 - loss: 59.3522 - crf_viterbi_accuracy: 0.7196

KeyboardInterrupt: ignored

In [0]:
model.save(addh + config.MODEL_PATH)